### Converting AVIRIS-NG ENVI File Fomat to netCDF

#### Resources
This is developed from content provided by Jack McNelis from a similar dataset set at PO.DAAC called PRISM<br>
- https://nbviewer.org/github/jjmcnelis/nasa-prism-data-decomposed/blob/master/PRISM-Data-Decomposed.ipynb

Additional information on gdal AVIRIS ENVI file format transformation is available here:<br>
- https://gis.stackexchange.com/questions/229952/rotate-envi-hyperspectral-imagery-with-gdal

####  In this example, I'm using an example dataset that is available from JPL, but also distributed from the ORNL DAAC as part of the ABoVE Project 

#### Specifically an L2 reflectance file from the flight path defined as `ang20170706t180635`
#### `https://avng.jpl.nasa.gov/avng/y17_data/ang20170706t180635rfl.tar.gz`

JPL Supporting Sites:<br>
2017 AVIRIS Next Generation Flights:  https://avirisng.jpl.nasa.gov/cgi/flights_17.cgi?step=view_all_flights<br>
AVIRIS-NG Flight: ang20170706t: https://avirisng.jpl.nasa.gov/cgi/flights_17.cgi?step=view_flightlog&flight_id=ang20170706t<br>

#### Michele's working directories (for my reference):
Local File:  `C:\DAAC\1_EVS\AVIRIS\AVIRIS-NG\JPL_download\ABoVE\` <br>
GridBGC Server: `//daymet_data/mytdaymet/AVIRIS-NG`

Quick check of the projection<br>
[daymet@gridbgc AVIRIS-NG]$ **gdalsrsinfo ang20170706t180635_corr_v2p9_img** <br>
PROJ.4 : +proj=utm +zone=6 +datum=WGS84 +units=m +no_defs <br>


### Requirements:
#### Community software requirements include:

`numpy` https://numpy.org/doc/stable/index.html -- numpy does most of everything except the coordinate transforms, even reads the binary file.

`netCDF4` https://unidata.github.io/netcdf4-python/ -- for writing a beautiful netCDF-4 file

`pyproj` https://pyproj4.github.io/pyproj/stable/ -- can't calculate UTMs reliably with flight lines spread across all latitudes. PROJ, tho.

In [1]:
import numpy as np
from netCDF4 import Dataset
from pyproj.transformer import AreaOfInterest, Transformer
from pyproj import crs

#### Python 3 standard library imports:

In [2]:
import tarfile
from io import TextIOWrapper
from datetime import datetime
from os.path import basename, isfile
from urllib.request import urlretrieve
from shutil import which

#### File is already downloaded to this path

In [3]:
path = "C:\\DAAC\\1_EVS\\AVIRIS\\AVIRIS-NG\\JPL_download\\ABoVE\\"
file = "ang20170706t180635rfl.tar.gz"
tfile = path + file
tfile

'C:\\DAAC\\1_EVS\\AVIRIS\\AVIRIS-NG\\JPL_download\\ABoVE\\ang20170706t180635rfl.tar.gz'

#### Contents of tar file

In [ ]:
# This is demonstrated for potential further use.  Not necessary for this draft code
with tarfile.open(tfile, "r") as z:
    tcontents = z.getnames()
tcontents

#### L2 Reflectance file is `ang20170706t180635_corr_v2p9_img` (no ext) & `.hdr`
Notice path is `//ang20170706t180635_rfl_v2p9/ang20170706t180635_corr_v2p9_img` (& `.hdr`)

#### Create out file for netCDF

In [ ]:
# not using in this draft
out_nc = tfile.replace(".tar.gz", ".nc")
out_nc

## Examine ENVI Header File for Reflectance data
https://www.harrisgeospatial.com/docs/enviheaderfiles.html

Read the ENVI Header file as text<br>
C:\DAAC\1_EVS\AVIRIS\AVIRIS-NG\JPL_download\ABoVE\ang20170706t180635_rfl_v2p9\ang20170706t180635_corr_v2p9_img.hdr

In [6]:
hdrpath = "C:\\DAAC\\1_EVS\\AVIRIS\\AVIRIS-NG\\JPL_download\\ABoVE\\ang20170706t180635_rfl_v2p9\\ang20170706t180635_corr_v2p9_img.hdr"
#C:\DAAC\1_EVS\AVIRIS\AVIRIS-NG\JPL_download\ABoVE\ang20170706t180635_rfl_v2p9
hdrpath

'C:\\DAAC\\1_EVS\\AVIRIS\\AVIRIS-NG\\JPL_download\\ABoVE\\ang20170706t180635_rfl_v2p9\\ang20170706t180635_corr_v2p9_img.hdr'

In [7]:
#with open("document.txt", "r", encoding='utf-8-sig') as f:
#    string = f.read()
with open(hdrpath, "r", encoding='utf-8-sig') as f:
    hdr = f.read()

In [8]:
# Split the hearder in  a list of lines.  Drop the first one.
hlines = hdr.split("\n")[1:]
hlines

['description = {',
 '  AVIRIS-NG Measured Radiances in uW nm-1 cm-2 sr-1}',
 'samples = 697',
 'lines = 6454',
 'bands = 425',
 'header offset = 0',
 'file type = ENVI',
 'data type = 4',
 'interleave = bil',
 'byte order = 0',
 'map info = { UTM , 1 , 1 , 464375.622302 , 7180773.22613 , 5.1 , 5.1 , 6 , North , WGS-84 , units=Meters , rotation=-82.0000000 }',
 'crosstrack scatter file = /home/winstono/isat-ang2017/ang/cal/data/20170125_via_ang20160925t182412_crf',
 'wavelength units = Nanometers',
 'flat field file = /home/winstono/isat-ang2017/ang/cal/data/20170320_ang20170313_BLUSS_avg_rows300-340_ff',
 'spectral scatter file = /home/winstono/isat-ang2017/ang/cal/data/20170125_via_ang20160925t182412_srf',
 'wavelength file = /home/winstono/isat-ang2017/ang/cal/data/20170320_ang20170228_wavelength_fit_full.txt',
 'wavelength = { 376.86 , 381.87 , 386.88 , 391.89 , 396.89 , 401.9 , 406.91 , 411.92 , 416.93 , 421.94 , 426.95 , 431.96 , 436.96 , 441.97 , 446.98 , 451.99 , 457.0 , 462.01

In [9]:
# Parsing header in nested loops.  First, split key from values.
hpairs = [l.split(" = ") for l in hlines if " = " in l]
hpairs

# Format the strings as a dictionary
#hdict = {l[0]: l[1].strip() for l in hpairs}
#hdict

[['description', '{'],
 ['samples', '697'],
 ['lines', '6454'],
 ['bands', '425'],
 ['header offset', '0'],
 ['file type', 'ENVI'],
 ['data type', '4'],
 ['interleave', 'bil'],
 ['byte order', '0'],
 ['map info',
  '{ UTM , 1 , 1 , 464375.622302 , 7180773.22613 , 5.1 , 5.1 , 6 , North , WGS-84 , units=Meters , rotation=-82.0000000 }'],
 ['crosstrack scatter file',
  '/home/winstono/isat-ang2017/ang/cal/data/20170125_via_ang20160925t182412_crf'],
 ['wavelength units', 'Nanometers'],
 ['flat field file',
  '/home/winstono/isat-ang2017/ang/cal/data/20170320_ang20170313_BLUSS_avg_rows300-340_ff'],
 ['spectral scatter file',
  '/home/winstono/isat-ang2017/ang/cal/data/20170125_via_ang20160925t182412_srf'],
 ['wavelength file',
  '/home/winstono/isat-ang2017/ang/cal/data/20170320_ang20170228_wavelength_fit_full.txt'],
 ['wavelength',
  '{ 376.86 , 381.87 , 386.88 , 391.89 , 396.89 , 401.9 , 406.91 , 411.92 , 416.93 , 421.94 , 426.95 , 431.96 , 436.96 , 441.97 , 446.98 , 451.99 , 457.0 , 462.

In [10]:
# Format the strings as a dictionary
hdict = {l[0]: l[1].strip() for l in hpairs}
hdict

{'description': '{',
 'samples': '697',
 'lines': '6454',
 'bands': '425',
 'header offset': '0',
 'file type': 'ENVI',
 'data type': '4',
 'interleave': 'bil',
 'byte order': '0',
 'map info': '{ UTM , 1 , 1 , 464375.622302 , 7180773.22613 , 5.1 , 5.1 , 6 , North , WGS-84 , units=Meters , rotation=-82.0000000 }',
 'crosstrack scatter file': '/home/winstono/isat-ang2017/ang/cal/data/20170125_via_ang20160925t182412_crf',
 'wavelength units': 'Nanometers',
 'flat field file': '/home/winstono/isat-ang2017/ang/cal/data/20170320_ang20170313_BLUSS_avg_rows300-340_ff',
 'spectral scatter file': '/home/winstono/isat-ang2017/ang/cal/data/20170125_via_ang20160925t182412_srf',
 'wavelength file': '/home/winstono/isat-ang2017/ang/cal/data/20170320_ang20170228_wavelength_fit_full.txt',
 'wavelength': '{ 376.86 , 381.87 , 386.88 , 391.89 , 396.89 , 401.9 , 406.91 , 411.92 , 416.93 , 421.94 , 426.95 , 431.96 , 436.96 , 441.97 , 446.98 , 451.99 , 457.0 , 462.01 , 467.02 , 472.02 , 477.03 , 482.04 , 48

In [11]:
# Parse 'map info' into a labeled array
hdict['map info'] = [k.strip() for k in hdict['map info'][1:-1].split(" , ")]
#print(hdict['map info'])

In [12]:
# Iterate over header elements that are spectral (wavelength) dependent
for k in ['wavelength', 'fwhm', 'smoothing factors', 'correction factors', 'bbl']:
    hdict[k] = [float(v.strip()) for v in hdict[k][1:-1].split(",")]
hdr = hdict.copy()
hdr

{'description': '{',
 'samples': '697',
 'lines': '6454',
 'bands': '425',
 'header offset': '0',
 'file type': 'ENVI',
 'data type': '4',
 'interleave': 'bil',
 'byte order': '0',
 'map info': ['UTM',
  '1',
  '1',
  '464375.622302',
  '7180773.22613',
  '5.1',
  '5.1',
  '6',
  'North',
  'WGS-84',
  'units=Meters',
  'rotation=-82.0000000'],
 'crosstrack scatter file': '/home/winstono/isat-ang2017/ang/cal/data/20170125_via_ang20160925t182412_crf',
 'wavelength units': 'Nanometers',
 'flat field file': '/home/winstono/isat-ang2017/ang/cal/data/20170320_ang20170313_BLUSS_avg_rows300-340_ff',
 'spectral scatter file': '/home/winstono/isat-ang2017/ang/cal/data/20170125_via_ang20160925t182412_srf',
 'wavelength file': '/home/winstono/isat-ang2017/ang/cal/data/20170320_ang20170228_wavelength_fit_full.txt',
 'wavelength': [376.86,
  381.87,
  386.88,
  391.89,
  396.89,
  401.9,
  406.91,
  411.92,
  416.93,
  421.94,
  426.95,
  431.96,
  436.96,
  441.97,
  446.98,
  451.99,
  457.0,
  4

Display the keys of the header dictionary

In [13]:
list(hdr.keys())

['description',
 'samples',
 'lines',
 'bands',
 'header offset',
 'file type',
 'data type',
 'interleave',
 'byte order',
 'map info',
 'crosstrack scatter file',
 'wavelength units',
 'flat field file',
 'spectral scatter file',
 'wavelength file',
 'wavelength',
 'radiance version',
 'fwhm',
 'rcc file',
 'smoothing factors',
 'data ignore value',
 'bad pixel map',
 'correction factors',
 'bbl']

In [14]:
print(hdr['map info'])

['UTM', '1', '1', '464375.622302', '7180773.22613', '5.1', '5.1', '6', 'North', 'WGS-84', 'units=Meters', 'rotation=-82.0000000']


### Shape
Determine the shape of the 3D gridded dataset from samples, bands, lines.  Convert to int

In [15]:
bands = int(hdr['bands'])
samples = int(hdr['samples'])
lines = int(hdr['lines'])

print( bands, samples, lines)

425 697 6454


### Interleave
Note the `interleave` types and the `dimension orders`. We need to reshape the giant 1-dimensional array that we read from the binary file 

In [16]:
native_shape = {
    'BSQ': (samples, lines, bands),  # Band Sequential
    'BIP': (bands, samples, lines),  # Band Interleave by Pixel
    'BIL': (lines, bands, samples),  # Band Interleave by Line
}[hdr['interleave'].upper()]

print(native_shape)

(6454, 425, 697)


### Data Type
See the type map in the table on the ENVI header documentation https://www.harrisgeospatial.com/docs/enviheaderfiles.html. Also see the type codes given in the numpy.dtypes documentation https://numpy.org/doc/stable/reference/arrays.dtypes.html (and here https://numpy.org/doc/stable/user/basics.types.html).

Determine the corresponding `numpy` data type of the binary array stored in the ENVI image file.

In [17]:
#data type in ENVI header is 4
data_types = {
    '1': np.uint8,    # Byte: 8-bit unsigned integer
    '2': np.int16,    # Integer: 16-bit signed integer
    '3': np.int32,    # Long: 32-bit signed integer
    '4': np.single,   # Floating-point: 32-bit single-precision
    '5': np.double,   # Double-precision: 64-bit double-precision floating-point
    '6': np.csingle,  # Complex: Real-imaginary pair of single-precision floating-point
    '9': np.cdouble,  # Double-precision complex: Real-imaginary pair of double precision floating-point
    '12': np.uint16,  # Unsigned integer: 16-bit
    '13': np.uint32,  # Unsigned long integer: 32-bit
    '14': np.int64,   # 64-bit long integer (signed)
    '15': np.uint64,  # 64-bit unsigned long integer (unsigned)
}

In [18]:
hdr['data type'] = data_types[hdr['data type']]
hdr['data type']

numpy.float32

### Byte Order
The `byte order` field conveys the order of the bytes in integer, long integer, 64-bit integer, unsigned 64-bit integer, floating point, double precision, and complex data types. Use one of the following:

0: little endian; (Host (Intel) in the Header Info dialog) is least significant byte first (LSF) data (DEC and MS-DOS systems).
1: big endian; (Network (IEEE) in the Header Info dialog) is most significant byte first (MSF) data (all other platforms).

Map the byte order to the appropriate numpy encoding (prefixed to the data type).

In [19]:
hbyteorder = {
    '0': "<",  # little-endian
    '1': ">",  # big-endian
}[hdr['byte order']]

hbyteorder

'<'

### Map Info
Label the `map info` data in the header data dictionary.

In [21]:
map_info_labels = {
    0:  ("Projection name", str),
    1:  ("Reference (tie point) pixel x location (in file coordinates)", int),
    2:  ("Reference (tie point) pixel y location (in file coordinates)", int),
    3:  ("Pixel easting", float),
    4:  ("Pixel northing", float),
    5:  ("x pixel size", float),
    6:  ("y pixel size", float),
    7:  ("Projection zone (UTM only)", int),
    8:  ("North or South (UTM only)", str),
    9:  ("Datum", str),
    10: ("Units", str),
    11: ("Rotation", lambda x: float(x.split("=")[1])),
}

In [22]:
print(hdr['map info'])

['UTM', '1', '1', '464375.622302', '7180773.22613', '5.1', '5.1', '6', 'North', 'WGS-84', 'units=Meters', 'rotation=-82.0000000']


In [23]:
hdr['map info'] = {v[0]: v[1](hdr['map info'][k]) for k, v in map_info_labels.items()}
hdr['map info']

{'Projection name': 'UTM',
 'Reference (tie point) pixel x location (in file coordinates)': 1,
 'Reference (tie point) pixel y location (in file coordinates)': 1,
 'Pixel easting': 464375.622302,
 'Pixel northing': 7180773.22613,
 'x pixel size': 5.1,
 'y pixel size': 5.1,
 'Projection zone (UTM only)': 6,
 'North or South (UTM only)': 'North',
 'Datum': 'WGS-84',
 'Units': 'units=Meters',
 'Rotation': -82.0}

### Add explaination

netCDF + CF's grid mappings spec give recommendations for writing coordinate variables and other metadata georeference the data. We need to make four arrays of spatial coordinates to conform to grid mapping spec:

- a 1d array of X coordinates in meters (UTM eastings),
- a 1d array of Y coordinates in meters (UTM northings),
- a 2d array of longitude coordinates in decimal degrees,
- a 2d array of latitude coordinates in decimal degrees,

Get the X,Y origin and resolution from the `map info` header field.

In [24]:
# XY pixel index of the raster origin.
xindex = hdr['map info']['Reference (tie point) pixel x location (in file coordinates)']
yindex = hdr['map info']['Reference (tie point) pixel y location (in file coordinates)']

# Get the X and Y position of the raster origin in meters.
xorigin = hdr['map info']['Pixel easting']
yorigin = hdr['map info']['Pixel northing']

# Get the X and Y dimensions of the pixels in meters.
xresolution = hdr['map info']['x pixel size']
yresolution = hdr['map info']['y pixel size']

(xorigin, xresolution, yorigin, yresolution)

(464375.622302, 5.1, 7180773.22613, 5.1)

### Add explaination

Important: Header gives raster rotation in degrees, but we need it in radians.<br>
(-82° × π/180 = -1.431rad)



In [25]:
rotation = (np.pi/180) * hdr['map info']['Rotation']
rotation

-1.4311699866353502

### Add explaination

Define the transformation using six coefficients. Because modified coefficients for items 1, 2, 4, and 5:

0. x origin      # (The origin refers to top-left corner of top-left pixel, in this case.)
1. x resolution 
2. x rotation
3. y origin
4. y rotation
5. y resolution

In [63]:
#gt = (
#    xorigin, 
#    np.cos(rotation)*xresolution,
#    np.sin(rotation)*xresolution,
#    yorigin, 
#    np.sin(rotation)*yresolution,
#    -np.cos(rotation)*yresolution, 
#)
#gt

#(464375.622302,
# 0.7097828148963339,
# -5.050367150582009,
# 7180773.22613,
# -5.050367150582009,
# -0.7097828148963339)

#updated this based on https://gis.stackexchange.com/questions/229952/rotate-envi-hyperspectral-imagery-with-gdal
gt = (
    xorigin, 
    np.cos(rotation)*xresolution,
    -np.sin(rotation)*xresolution,
    yorigin, 
    np.sin(rotation)*yresolution,
    np.cos(rotation)*yresolution, 
)
gt


(464375.622302,
 0.7097828148963339,
 5.050367150582009,
 7180773.22613,
 -5.050367150582009,
 0.7097828148963339)

Get the six coefficients like GDAL does:

#### Confirm with gdal

[daymet@gridbgc AVIRIS-NG]$ `gdalinfo ang20170706t180635_corr_v2p9_img | grep -A 2 GeoTransform`

Returns identical transformation coefficients:

GeoTransform =<br>
  464375.622302, 0.7097828148963339, -5.050367150582009<br>
  7180773.22613, -5.050367150582009, -0.7097828148963339<br>

  INTERLEAVE=LINE<br>
Corner Coordinates:<br>
Upper Left  (  464375.622, **7180773.226**) (147d44'54.48"W, 64d44'59.02"N)<br>
Lower Left  (  **431780.553**, 7176192.288) (148d25'51.29"W, 64d42'12.96"N)<br>
Upper Right (  **464870.341**, 7177253.120) (147d44'13.96"W, 64d43' 5.51"N)<br>
Lower Right (  432275.271, **7172672.182**) (148d25' 8.00"W, 64d4<br>
  
#### ESRI Reports the following on the ENVI file - which is spatially correct
Top: 7180773.22613 <br>
Left: 431780.552712 <br>
Right: 464870.340924 <br>
Bottom: 7172672.18194 <br>

The new coefficients apply the correct transform, yielding native UTM coordinates to compensate for our rotated grid.

### Lots!! to explain these steps in Jack's code

Print the first/last items in each of the coordinate arrays + their sizes.

In [64]:
#x = np.array([gt[0]+i*gt[1] for i in range(0, samples)])
#y = np.array([gt[3]+i*gt[5] for i in range(0, lines)])

x = np.array([gt[0]+i*gt[2] for i in range(0, samples)])
y = np.array([gt[3]+i*gt[4] for i in range(0, lines)])
print(f"X ({len(x)}):  {round(x[0], 5)} -  {round(x[-1], 5)}", "\n"
      f"Y ({len(y)}): {round(y[0], 5)} - {round(y[-1], 5)}")

#X (697):  464375.6223 -  460860.56677 
#Y (6454): 7180773.22613 - 7148183.20691

X (697):  464375.6223 -  467890.67784 
Y (6454): 7180773.22613 - 7148183.20691


*WHY?* Y coordinates should not descend. We need to flip the array, and we may need to flip the entire dataset along the Y dimension, too.

In [65]:
y = np.flip(y)

y[0], y[-1]

(7148183.206907295, 7180773.22613)

The first and last values in the new X and Y arrays are shown above in meters (UTM eastings and northings). But we actually need to shift them by one half pixel in both directions, like XUTM+0.5*XRES, YUTM+0.5*YRES. This will shift the Xs and Ys to the pixel centers. They were at the top left corners of the pixels before.

In [ ]:
#x = [i+0.5*gt[1] for i in x]
#y = [j+0.5*gt[5] for j in y]

#x[0], y[0]

In [66]:
#x = [i+0.5*gt[1] for i in x]
#y = [j+0.5*gt[5] for j in y]

x = [i+0.5*gt[2] for i in x]
y = [j+0.5*gt[4] for j in y]

x[0], y[0]

(464378.1474855753, 7148180.681723719)

To prepare for the next step, get two 2-dimensional arrays of X and Y coordinates by expanding column- and row-wise to reference every pixel.

In [67]:
x2d, y2d = np.meshgrid(x, y)

x2d.shape

(6454, 697)

***longitudes, latitudes***

Now we need to get two 2-dimensional arrays of longitudes and latitudes that coincide with the permuted X,Y positions. Here's the proj4 string that represents the appropriate UTM zone and north/south hemisphere.

In [68]:
zone = hdr['map info']['Projection zone (UTM only)']
hemi = hdr['map info']['North or South (UTM only)']

# Format the proj4 string with the UTM zone and the ns hemisphere identifier.
proj4 = f"+proj=utm +zone={zone} +{hemi.lower()} +datum=WGS84 +units=m +no_defs"

# Neatly print the UTM zone information and its proj4 string.
print(f"UTM {zone} {hemi}:  '{proj4}'")

UTM 6 North:  '+proj=utm +zone=6 +north +datum=WGS84 +units=m +no_defs'


In [69]:
#epsg = f"326{zone}" if hemi is "+north" else f"327{zone}"
# not working and not including the 0 in Zone 06
# Hardcoded for now
epsg = 32606
epsg

32606

In [70]:
# Init the transform based on source and target projections.
xform = Transformer.from_crs(f"epsg:{epsg}", "epsg:4326", always_xy=True)

# Aply PROJ default transform utm>>geo to all X,Y coordinates.
lon, lat = xform.transform(x2d, y2d)

lon.shape

(6454, 697)

In [71]:
print(lon.min(), lat.min(), lon.max(), lat.max())

-147.74841197056503 64.45729134375458 -147.66736309245746 64.75006015113453


## Read Image

In [73]:
imgpath = "C:\\DAAC\\1_EVS\\AVIRIS\\AVIRIS-NG\\JPL_download\\ABoVE\\ang20170706t180635_rfl_v2p9\\ang20170706t180635_corr_v2p9_img"
#C:\DAAC\1_EVS\AVIRIS\AVIRIS-NG\JPL_download\ABoVE\ang20170706t180635_rfl_v2p9
imgpath

'C:\\DAAC\\1_EVS\\AVIRIS\\AVIRIS-NG\\JPL_download\\ABoVE\\ang20170706t180635_rfl_v2p9\\ang20170706t180635_corr_v2p9_img'

In [74]:
with open(imgpath, "r") as rflimg:
    arr = np.fromfile(rflimg, dtype=hdr['data type'] )

In [75]:
arr

array([-9.9990000e+03,  3.4583617e-02, -9.9990000e+03, ...,
       -9.9990000e+03, -9.9990000e+03, -9.9990000e+03], dtype=float32)

In [76]:
arr.size

1911836150

#### arr.size should equal the samples * bands * lines from the ENVI image header

In [77]:
samples*bands*lines

1911836150

#### Reshape the array to match dimensions ordered for BIL.

In [78]:
arr = arr.reshape(native_shape)
arr.shape

(6454, 425, 697)

In [79]:
arr = arr.transpose((1, 0, 2))
arr.shape

(425, 6454, 697)

#### Flip the raster on its Y axis similar to flipping the Y coordinates

In [80]:
arr = arr[:, ::-1, :]

## Write netCDF-4 output

Open the dataset for writing

In [81]:
outpath = "C:\\DAAC\\1_EVS\\AVIRIS\\AVIRIS-NG\\JPL_download\\ABoVE\\ang20170706t180635_rfl_v2p9\\"
dsrfl = Dataset(outpath + 'ang20170706t180635_rfl_v2p9_img.nc', 'w', clobber=True, format='NETCDF4')

In [82]:
#dsrfl.close()
dsrfl

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 

#### Make a dictionary to store the global attributes

In [83]:
atts = dict(
    id                       = "10.3334/AVIRIS_L2/#",
    naming_authority         = "daac.ornl.gov",
    license                  = "https://science.nasa.gov/earth-science/earth-science-data/data-information-policy/",
    project                  = "NASA AVIRIS Facility Instrument",
    project_url              = "https://daac.ornl.gov/cgi-bin/dataset_lister.pl?p=3##",
    institution              = "ORNL DAAC",
    instrument               = "AVIRIS-NG (Airborne Visible Infrared Imaging Spectrometer - Next Generation)",
    platform                 = "varies",
    Conventions              = "CF-1.7",
    keywords_vocabulary      = "GCMD Science Keywords",
    standard_name_vocabulary = "CF Standard Names v72",
    processing_version       = "v2p9",
    product_version          = "Radiance version v2.0",
    product_name             = "ang20170706t180635_rfl_v2p9_img.nc",
)

In [84]:
atts['geospatial_lon_min']   = lon.min()
atts['geospatial_lon_max']   = lon.max()
atts['geospatial_lon_units'] = "degrees_east"
atts['geospatial_lat_min']   = lat.min()
atts['geospatial_lat_max']   = lat.max()
atts['geospatial_lat_units'] = "degrees_north"

#### Jack has more sample CF and ACDD Convension metadata - see example

### dimensions

samples and lines (both integers) are used to specify the size of x and y dimension, respectively, in the output file

In [85]:
xdim = dsrfl.createDimension('x', size=samples)
ydim = dsrfl.createDimension('y', size=lines)
bdim = dsrfl.createDimension('band', size=bands)

dsrfl

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): x(697), y(6454), band(425)
    variables(dimensions): 
    groups: 

### coordinate variables

Add the `x` and `y` coordinates and attributes as new variables.

In [86]:
x_var = dsrfl.createVariable('x', 'f8', ('x'), fill_value=None)
x_var.units = "m"
x_var.axis = "X"
x_var.standard_name = "projection_x_coordinate"
x_var.long_name = "x coordinate of projection"
x_var[:] = x

y_var = dsrfl.createVariable('y', 'f8', ('y'), fill_value=None)
y_var.units = "m"
y_var.axis = "Y"
y_var.standard_name = "projection_y_coordinate"
y_var.long_name = "y coordinate of projection"
y_var[:] = y

dsrfl

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): x(697), y(6454), band(425)
    variables(dimensions): float64 x(x), float64 y(y)
    groups: 

The longitude and latitude arrays are 2-dimensional.  Make sure to order to `y,x`

In [87]:
lat_var = dsrfl.createVariable('lat', 'f4', ('y', 'x'), fill_value=None)
lat_var.units = "degrees_north"
lat_var.standard_name = "latitude"
lat_var.long_name = "latitude"
lat_var[:,:] = lat

lon_var = dsrfl.createVariable('lon', 'f4', ('y', 'x'), fill_value=None)
lon_var.units = "degrees_east"
lon_var.standard_name = "longitude"
lon_var.long_name = "longitude"
lon_var[:,:] = lon

dsrfl

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): x(697), y(6454), band(425)
    variables(dimensions): float64 x(x), float64 y(y), float32 lat(y, x), float32 lon(y, x)
    groups: 

#### Some CF Grid Mapping
Use the transverse_mercator standard grid mapping (CF-1.6+) and pyproj https://nbviewer.org/github/jjmcnelis/nasa-prism-data-decomposed/blob/master/PRISM-Data-Decomposed.ipynb#.  This can dump the CF standard attributes as a Python dictionary.  

Grid mapping spec clean format here:  https://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/build/apf.html

In [88]:
grid_mapping_atts = crs.CRS(proj4).to_cf()
grid_mapping_atts

{'crs_wkt': 'PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ID["EPSG",6326]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8901]]],CONVERSION["UTM zone 6N",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",-147,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Scale factor at natural origin",0.9996,SCALEUNIT["unity",1],ID["EPSG",8805]],PARAMETER["False easting",500000,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]],ID["EPSG",16006]],CS[Cartesian,2],AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["(N)",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]',
 'semi_major_axis': 6378137.0,
 'semi_minor_axis': 6356752.314245179,
 'inverse_flattening

Create the CF-required grid_mapping variable and set those attributes

In [89]:
crs_var = dsrfl.createVariable("UTM_Projection", "|S1")
crs_var.setncatts(grid_mapping_atts)
#crs_var._CoordinateTransformType = "Projection";
#crs_var._CoordinateAxisTypes = "GeoY GeoX";

crs_var

<class 'netCDF4._netCDF4.Variable'>
|S1 UTM_Projection()
    crs_wkt: PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ID["EPSG",6326]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8901]]],CONVERSION["UTM zone 6N",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",-147,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Scale factor at natural origin",0.9996,SCALEUNIT["unity",1],ID["EPSG",8805]],PARAMETER["False easting",500000,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]],ID["EPSG",16006]],CS[Cartesian,2],AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["(N)",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]
    semi_major_axis: 6378137.0
    

#### bands

The band dimension band coordinates com efrom the ENVI header's wavelength field.  A few other sensor-related variables.

In [90]:
# Get the array of wavelengths taken from the header.
wave = np.array(hdr['wavelength'])

# Create a wavelength variable; add the attributes and data.
wave_var = dsrfl.createVariable('wavelength', 'f4', ('band'), fill_value=None)
wave_var.long_name = "wavelengths of band centers"
wave_var.units = "nm"
wave_var[:] = wave

wave_var

<class 'netCDF4._netCDF4.Variable'>
float32 wavelength(band)
    long_name: wavelengths of band centers
    units: nm
unlimited dimensions: 
current shape = (425,)
filling on, default _FillValue of 9.969209968386869e+36 used

#### data

In [91]:
#Ensure fill value parsed from ENVI header is same dtype as the data array
data_type = hdr['data type']
fill_value = data_type( hdr['data ignore value'] )

data = dsrfl.createVariable('reflectance', 'f4', ('band', 'y', 'x'), zlib=True, complevel=4, fill_value=fill_value)
data.long_name = "reflectance"
data.units = "unitless"
data.grid_mapping = "UTM_Projection"
data.coordinates = "lat lon"

data


<class 'netCDF4._netCDF4.Variable'>
float32 reflectance(band, y, x)
    _FillValue: -9999.0
    long_name: reflectance
    units: unitless
    grid_mapping: UTM_Projection
    coordinates: lat lon
unlimited dimensions: 
current shape = (425, 6454, 697)
filling on

In [92]:
data[:,:,:] = arr #.reshape((bands, lines, samples))
data

<class 'netCDF4._netCDF4.Variable'>
float32 reflectance(band, y, x)
    _FillValue: -9999.0
    long_name: reflectance
    units: unitless
    grid_mapping: UTM_Projection
    coordinates: lat lon
unlimited dimensions: 
current shape = (425, 6454, 697)
filling on

In [93]:
dsrfl.setncatts(atts)

In [94]:
dsrfl.close()